In [ ]:
import pyautogui as pg
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier


# %matplotlib inline
import matplotlib.pyplot as plt
from scipy import misc
import skimage.measure

import os
from time import sleep

# Globals

H = 1600
W = 2560 

NUM_CROP_W = 82
NUM_CROP_H = 86

ROUND_CROP_W = 154
ROUND_CROP_H = 86

numbs_coords = {
    0: {"x": 122, "y": 116, "w": NUM_CROP_W, "h": NUM_CROP_H},
    1: {"x": 338, "y": 116, "w": NUM_CROP_W, "h": NUM_CROP_H},
    2: {"x": 865, "y": 116, "w": NUM_CROP_W, "h": NUM_CROP_H},
}
wins_coords ={
    0: {"x": 552, "y": 116, "w": ROUND_CROP_W, "h": ROUND_CROP_H},
}


# 0 - loading_screen
# 1 - name_team
# 2 - main_game
# 3 - clickthrough
# 4 - excess_gold



In [ ]:
anifoods = [
"none",
"empty",
"ant",
"apple",
"badgr",
"beavr",
"bfish",
"bison",
"boar",
"camel",
"can",
"cat",
"ccake",
"chili",
"choco",
"cow",
"crab",
"crikt",
"croc",
"deer",
"dlphn",
"dodo",
"dog",
"dragn",
"duck",
"elpht",
"fish",
"fly",
"garlc",
"goril",
"grafe",
"hghog",
"hippo",
"honey",
"horse",
"kanga",
"leprd",
"mamth",
"meat",
"melon",
"milk",
"mingo",
"monky",
"mosqt",
"mushm",
"otter",
"ox",
"parrt",
"pcock",
"pear",
"pguin",
"pig",
"pill",
"pizza",
"rabit",
"rat",
"rhino",
"roost",
"salad",
"scorp",
"seal",
"shark",
"sheep",
"shrmp",
"skunk",
"snail",
"snake",
"spidr",
"sqrrl",
"steak",
"sushi",
"swan",
"tiger",
"turky",
"turtl",
"whale",
"worm"
]
values = range(-1, len(anifoods))

anifood_map = dict(zip(anifoods, values))

In [ ]:
# Utility functions
def show_cv2_img(window_name, img):
    cv2.imshow(window_name, img)
    cv2.waitKey()
    cv2.destroyAllWindows()


def img_info(ndarray, gray=0):
    '''displays image and shape; gray is a flag used by matplotlib'''
    plt.figure(figsize=(18,8))
    if gray:
        plt.imshow(ndarray, cmap = plt.get_cmap('gray'))
    else:
        plt.imshow(ndarray);
    print('Image shape:', ndarray.shape)


In [ ]:
steps = '''
Steps for generating data:
    read in image,
    cut all relevent regions, --> make parameter
    for each region,
        convert region to BGR,
        convert region to greyscale,
        standardize region,
        print region,
        label region,
        add region to X,
        add label to Y
'''

In [ ]:

def load_files(dir=None):
    if dir is None:
        file_dir = "training_base"
    else:
        file_dir = dir
    # load list of files
    file_list = []
    for base, dirs, files in os.walk(file_dir):
        for f in files:
            if not f.startswith("."):
                file_list.append(os.path.join(base, f)) 
    return file_list

def int_input(msg):
    label = None
    try:
        label = int(input(msg))
        if label in range(-1, 20):
            return True, label
        else:
            return False, -2
    except:
        return False, -2

def get_regions(img, coords):
    regions = []
    for k, v in coords.items():
        x = v["x"]
        y = v["y"]
        w = v["w"]
        h = v["h"]

        # Sanity check
        # print(f"(Y: {y}, H: {h})  (X: {x}, W: {w})")
        region = img[y: y + h, x: x + w]
        regions.append(region)
        
        # Sanity check 2
        # img_info(region, gray=1)
    return regions

        


def gen_training(file_list, coords):
    # Output arrays for training data
    X = None
    y = None
    
    iter = 0
    # iterate over all files and classify them
    for fname in file_list:
        print(f"File name: {fname}")
        iter += 1
        base_img = cv2.imread(fname, cv2.COLOR_RGB2BGR)    # read in base img from file
        
        # #TODO: Cut the relevent regions out here, for now just base_img is 1 region
        regions = get_regions(base_img, coords)

        for img in regions:

            img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)    # Convert to BGR
            # show_cv2_img("ORIG", img)
            img = np.dot(img, [0.2989, 0.5870, 0.1140])             # Convert to grayscale
            # show_cv2_img("GRAYSCALE", img)
            img = skimage.measure.block_reduce(img, (2,2), np.max)  # Downsample
            # show_cv2_img("DOWNSAMPLE", img)
            img = (img - img.mean()) / np.sqrt(img.var() + 1e-5)    # Rescale
            # show_cv2_img("RESCALED", img)

            # Show region and ask to label
            show_cv2_img(f"Example: {iter}", img)

            # Get user input 
            isOK, label = int_input("\rLabel? ")
            while isOK != True:
                print(f"\rNOt OK! isOK: {isOK}")
                print(f"\rlabel is: {label}")
                isOK, label = int_input("\rMust be int range(0, 20) Label?: ")
 
            if label == -1:
                print("Okay! We're done here.")
                print(f"X is:\n{X}")
                print(f"y is:\n{y}")
                return None, None

            
            print("\rAdding to X and y...")
            if y is None:
                y = np.array(label)
            else:
                y = np.append(y, label)

            if X is None:
                X = np.array(img.flatten())
            else:
                X = np.vstack((X, img.flatten()))
            
    return X, y


def add_win_training(file_list, coords):
    # Output arrays for training data
    X = None
    y = None
    
    iter = 0
    total = len(flist)
    # iterate over all files and classify them
    for fname in file_list:
        ftype = os.path.basename(fname).split("_")[0]
        if ftype != "lives":
            continue

        iter += 1
        print(f"Iter: {iter}/{total}")
        base_img = cv2.imread(fname, cv2.COLOR_RGB2BGR)    # read in base img from file
        
        # #TODO: Cut the relevent regions out here, for now just base_img is 1 region
        regions = get_regions(base_img, coords)

        for img in regions:

            img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)    # Convert to BGR
            # show_cv2_img("ORIG", img)
            img = np.dot(img, [0.2989, 0.5870, 0.1140])             # Convert to grayscale
            # show_cv2_img("GRAYSCALE", img)
            img = skimage.measure.block_reduce(img, (2,2), np.max)  # Downsample
            # show_cv2_img("DOWNSAMPLE", img)
            img = (img - img.mean()) / np.sqrt(img.var() + 1e-5)    # Rescale
            # show_cv2_img("RESCALED", img)

            label = 9

            
            print("\rAdding to X and y...")
            if y is None:
                y = np.array(label)
            else:
                y = np.append(y, label)

            if X is None:
                X = np.array(img.flatten())
            else:
                X = np.vstack((X, img.flatten()))
            
    return X, y

def gen_situation_training(file_list):
    # 0 - LOADING
# 1 - NAME TEAM
# 2 - MAIN GAME
# 3 - CLICK THROUGH
# 4 - EXCESS GOLD1

    label_map = dict(
        loading      = 0,
        name         = 1,
        main         = 2,
        clickthrough = 3,
        excess       = 4
    )

    # Output arrays for training data
    X = None
    y = None
    
    iter = 0
    todo = len(file_list)
    # iterate over all files and classify them
    for fname in file_list:
        print(f"File name: {fname}")
        iter += 1
        print(f"iter {iter}/{todo}")
        if iter == 500:
            break
        img = cv2.imread(fname, cv2.COLOR_RGB2BGR)    # read in base img from file
        

        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)    # Convert to BGR
        # show_cv2_img("ORIG", img)
        img = np.dot(img, [0.2989, 0.5870, 0.1140])             # Convert to grayscale
        # show_cv2_img("GRAYSCALE", img)
        img = skimage.measure.block_reduce(img, (8,8), np.max)  # Downsample
        # show_cv2_img("DOWNSAMPLE", img)
        img = (img - img.mean()) / np.sqrt(img.var() + 1e-5)    # Rescale
        # show_cv2_img("RESCALED", img)
        print(img.shape)

        file_lname = os.path.basename(fname).split("_")[0]
        label = label_map[file_lname]
        
        print(f"Adding to X and y. label - {file_lname}: {label}")
        if y is None:
            y = np.array(label)
        else:
            y = np.append(y, label)

        if X is None:
            X = np.array(img.flatten())
        else:
            X = np.vstack((X, img.flatten()))
            
    return X, y


def gen_numb_training2(file_list):
    # Output arrays for training data
    X = None
    y = None
    
    iter = 0
    todo = len(file_list)
    # iterate over all files and classify them
    for fname in file_list:
        label = int(os.path.basename(fname).split("_")[0])
        iter += 1
        print(f"iter {iter}/{todo}")

        img = cv2.imread(fname, cv2.COLOR_RGB2BGR)    # read in base img from file
        

        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)    # Convert to BGR
        # # show_cv2_img("ORIG", img)
        img = np.dot(img, [0.2989, 0.5870, 0.1140])             # Convert to grayscale
        # # show_cv2_img("GRAYSCALE", img)
        img = skimage.measure.block_reduce(img, (2,2), np.max)  # Downsample
        # # show_cv2_img("DOWNSAMPLE", img)
        img = (img - img.mean()) / np.sqrt(img.var() + 1e-5)    # Rescale
        # show_cv2_img("RESCALED", img)
        # # print(img.shape)

        
        if y is None:
            y = np.array(label)
        else:
            y = np.append(y, label)

        if X is None:
            X = np.array(img.flatten())
        else:
            X = np.vstack((X, img.flatten()))
            
    return X, y


def gen_anifood_training(file_list):
    # Output arrays for training data
    X = None
    y = None
    
    iter = 0
    todo = len(file_list)
    # iterate over all files and classify them
    for fname in file_list:
        aniname = os.path.basename(fname).split("_")[0]
        label = anifood_map[aniname]
        # print(f"{aniname}: {label}")
        iter += 1
        if iter % 50 == 0:
            print(f"iter {iter}/{todo}")

        img = cv2.imread(fname, cv2.COLOR_RGB2BGR)    # read in base img from file
        

        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)    # Convert to BGR
        img = np.dot(img, [0.2989, 0.5870, 0.1140])             # Convert to grayscale
        img = skimage.measure.block_reduce(img, (2,2), np.max)  # Downsample
        img = (img - img.mean()) / np.sqrt(img.var() + 1e-5)    # Rescale
        # show_cv2_img("RESCALED", img)
        
        if y is None:
            y = np.array(label)
        else:
            y = np.append(y, label)

        if X is None:
            X = np.array(img.flatten())
        else:
            X = np.vstack((X, img.flatten()))
            
    return X, y


def gen_attack_or_health_training(file_list):
    # Output arrays for training data
    X = None
    y = None
    
    iter = 0
    todo = len(file_list)
    # iterate over all files and classify them
    for fname in file_list:
        label = int(os.path.basename(fname).split("_")[0])
        # label = anifood_map[aniname]
        # print(f"Label: {label}")
        iter += 1
        if iter % 50 == 0:
            print(f"iter {iter}/{todo}")

        img = cv2.imread(fname, cv2.COLOR_RGB2BGR)    # read in base img from file
        

        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)    # Convert to BGR
        img = np.dot(img, [0.2989, 0.5870, 0.1140])             # Convert to grayscale
        img = skimage.measure.block_reduce(img, (2,2), np.max)  # Downsample
        img = (img - img.mean()) / np.sqrt(img.var() + 1e-5)    # Rescale
        # show_cv2_img("RESCALED", img)
        
        if y is None:
            y = np.array(label)
        else:
            y = np.append(y, label)

        if X is None:
            X = np.array(img.flatten())
        else:
            X = np.vstack((X, img.flatten()))
            
    return X, y





In [40]:
flist = load_files("health_training")
print(len(flist))
X, y = gen_attack_or_health_training(flist)

11050
iter 50/11050
iter 100/11050
iter 150/11050
iter 200/11050
iter 250/11050
iter 300/11050
iter 350/11050
iter 400/11050
iter 450/11050
iter 500/11050
iter 550/11050
iter 600/11050
iter 650/11050
iter 700/11050
iter 750/11050
iter 800/11050
iter 850/11050
iter 900/11050
iter 950/11050
iter 1000/11050
iter 1050/11050
iter 1100/11050
iter 1150/11050
iter 1200/11050
iter 1250/11050
iter 1300/11050
iter 1350/11050
iter 1400/11050
iter 1450/11050
iter 1500/11050
iter 1550/11050
iter 1600/11050
iter 1650/11050
iter 1700/11050
iter 1750/11050
iter 1800/11050
iter 1850/11050
iter 1900/11050
iter 1950/11050
iter 2000/11050
iter 2050/11050
iter 2100/11050
iter 2150/11050
iter 2200/11050
iter 2250/11050
iter 2300/11050
iter 2350/11050
iter 2400/11050
iter 2450/11050
iter 2500/11050
iter 2550/11050
iter 2600/11050
iter 2650/11050
iter 2700/11050
iter 2750/11050
iter 2800/11050
iter 2850/11050
iter 2900/11050
iter 2950/11050
iter 3000/11050
iter 3050/11050
iter 3100/11050
iter 3150/11050
iter 3

In [48]:
flist = load_files("attack_training")
print(len(flist))
X, y = gen_attack_or_health_training(flist)

10965
iter 50/10965
iter 100/10965
iter 150/10965
iter 200/10965
iter 250/10965
iter 300/10965
iter 350/10965
iter 400/10965
iter 450/10965
iter 500/10965
iter 550/10965
iter 600/10965
iter 650/10965
iter 700/10965
iter 750/10965
iter 800/10965
iter 850/10965
iter 900/10965
iter 950/10965
iter 1000/10965
iter 1050/10965
iter 1100/10965
iter 1150/10965
iter 1200/10965
iter 1250/10965
iter 1300/10965
iter 1350/10965
iter 1400/10965
iter 1450/10965
iter 1500/10965
iter 1550/10965
iter 1600/10965
iter 1650/10965
iter 1700/10965
iter 1750/10965
iter 1800/10965
iter 1850/10965
iter 1900/10965
iter 1950/10965
iter 2000/10965
iter 2050/10965
iter 2100/10965
iter 2150/10965
iter 2200/10965
iter 2250/10965
iter 2300/10965
iter 2350/10965
iter 2400/10965
iter 2450/10965
iter 2500/10965
iter 2550/10965
iter 2600/10965
iter 2650/10965
iter 2700/10965
iter 2750/10965
iter 2800/10965
iter 2850/10965
iter 2900/10965
iter 2950/10965
iter 3000/10965
iter 3050/10965
iter 3100/10965
iter 3150/10965
iter 3

In [ ]:
flist = load_files("anifood_training")
print(len(flist))
X, y = gen_anifood_training(flist)

In [ ]:
# Generate the NUMBS AI
flist = load_files("numbs_training")
print(len(flist))
X, y = gen_numb_training2(flist)

In [ ]:
# Generate Situation AI
flist = load_files("situation_training")
print(len(flist))
X, y = gen_situation_training(flist)


In [ ]:
# Generate the WINS AI
flist = load_files()
print(len(flist))
X, y = gen_training(flist, wins_coords)

In [ ]:
# Add to WINS AI
flist = load_files()
print(len(flist))
X, y = add_win_training(flist, wins_coords)

In [ ]:
# Print some initial stats about X and y
def print_counts(y):
    vals, counts = np.unique(y, return_counts=True)
    for val, count in zip(vals, counts):
        print(f"Val: {val}, Count: {count}")

# NOTE:
# Weak number in NUMBS that need a boost!  i.e. < 60 examples
# [0, 1, 2, 4, 5, 8, 11, 12, 13, 14]
# print(X.shape)
# print(y.shape)


In [ ]:
print_counts(y)

In [49]:
# Save the training data to disk

ml_data = "ml_data"
np.savetxt(os.path.join(ml_data, "attack_x_train.txt"), X)
np.savetxt(os.path.join(ml_data, "attack_y_train.txt"), y)

In [ ]:
# LOGISTIC REGRESSION STUFFSSS

model = LogisticRegression(multi_class="multinomial", max_iter=400)
solvers = ['lbfgs']
penalty = ['l2']
c_values = np.logspace(-2,0,5)

# # define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(
    estimator=model, 
    param_grid=grid, 
    n_jobs=-1, 
    cv=cv, 
    scoring='accuracy',
    error_score=0, 
    verbose=4,
)
grid_result = grid_search.fit(X, y)

In [ ]:
# # summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means  = grid_result.cv_results_['mean_test_score']
stds   = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


# Best: 0.998788 using {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}

# L2_grid_search_df = pd.DataFrame.from_dict(grid_result.cv_results_)
# L2_grid_search_df.to_csv("result_csvs/L2_grid_search_results.csv")

In [50]:
# Train the best model
# X = np.loadtxt(os.path.join("ml_data", "wins_x_train.txt"))
# y = np.loadtxt(os.path.join("ml_data", "wins_y_train.txt"))
best_attack_model = LogisticRegression(
    penalty="l2",
    solver="lbfgs",
    C=0.01,
    max_iter=700
    )

best_attack_model.fit(X, y)


LogisticRegression(C=0.01, max_iter=700)

In [51]:
result = best_attack_model.score(X, y)
print(result)

1.0


In [ ]:
# Lets try MLP

# Things to modify
# * Alpha for sure
# * Random state, since class
# * Activation function 

# X = np.loadtxt(os.path.join("ml_data", "numbs_x_train.txt"))
# y = np.loadtxt(os.path.join("ml_data", "numbs_y_train.txt"))

# print_counts(y)

from sklearn.neural_network import MLPClassifier
mlp_model         = MLPClassifier(max_iter=500)
mlp_solver        = ['lbfgs']
# mlp_random_states = [1,2,3,4,5]
mlp_activation_functions = ["logistic", "tanh", "relu"]
alphas = np.logspace(-2, 0, 5)

# define grid search
mlp_grid = dict(
    solver=mlp_solver,
    # random_state=mlp_random_states, 
    activation=mlp_activation_functions, 
    alpha=alphas, 
)
mlp_cv          = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
mlp_grid_search = GridSearchCV(
    estimator=mlp_model, 
    param_grid=mlp_grid, 
    n_jobs=-1, 
    cv=mlp_cv, 
    scoring='accuracy',
    error_score=0,
    verbose=4
)
mlp_grid_result = mlp_grid_search.fit(X, y)


In [ ]:
# # summarize results
print("Best: %f using %s" % (mlp_grid_result.best_score_, mlp_grid_result.best_params_))
means  = mlp_grid_result.cv_results_['mean_test_score']
stds   = mlp_grid_result.cv_results_['std_test_score']
params = mlp_grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [53]:
# Train Final Best MLP Model
# Best: 0.998788 using {'activation': 'logistic', 'alpha': 0.01, 'random_state': 1, 'solver': 'lbfgs'}

best_attack_MLP = MLPClassifier(
    activation   = "logistic",
    alpha        = 0.01,
    random_state = 1,
    solver       = "lbfgs",
    max_iter     = 500
    )

best_attack_MLP.fit(X, y)




MLPClassifier(activation='logistic', alpha=0.01, max_iter=500, random_state=1,
              solver='lbfgs')

In [54]:
score = best_attack_MLP.score(X, y)
print(score)

1.0


In [ ]:
predy = best_attack_MLP.predict(X)

In [ ]:
predy = best_attack_model.predict(X)

for i, group in enumerate(zip(y, predy)):
    if group[0] != group[1]:
        print(f"Pic #: {i}, Actual: {group[0]}, Pred: {group[1]}")

In [55]:
import pickle


# save the model to disk
filename = os.path.join("models", "attack_MLP_model.02")
pickle.dump(best_attack_MLP, open(filename, 'wb'))

# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X[0:100], y[:100])
print(result)







1.0


In [ ]:
# testx = np.loadtxt(os.path.join("ml_data", "numbs_x_train.txt"))
# testx.shape

##GET COORDS FUNCTION

In [ ]:
# find the coords for each attribute
# find in specific order:
# [coins, lives, wins, round]
# info_img_dir = "find_info_imgs"

# coins_file = os.path.join(info_img_dir, "coins.png")
# lives_file = os.path.join(info_img_dir, "lives.png")
# wins_file  = os.path.join(info_img_dir, "wins.png")
# round_file = os.path.join(info_img_dir, "round.png")

# base_file  = os.path.join(info_img_dir, "base_w_squares.png")

# target_files = [coins_file, lives_file, wins_file, round_file]




# for i, t in enumerate(target_files):
#     base_img = cv2.imread(base_file, cv2.COLOR_RGB2BGR)
#     search_img = cv2.imread(t, cv2.COLOR_RGB2BGR)

#     if i != 2:
#         w = NUM_CROP_W
#         h = NUM_CROP_H
#     else:
#         w = ROUND_CROP_W
#         h = ROUND_CROP_H

#     a, b, c, best_match_xy = cv2.minMaxLoc(cv2.matchTemplate(base_img, search_img, cv2.TM_CCOEFF_NORMED))
#     print(a, b, c, best_match_xy)
#     cv2.rectangle(base_img, (best_match_xy[0], best_match_xy[1]), (best_match_xy[0] + w, best_match_xy[1] + h), (0,255,255), 2)
#     show_cv2_img("bleh", base_img)



(122, 116)
(338, 116)
(552, 116)
(865, 115)